In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

# підключаємо бібліотеки для роботи із selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

# підключаємо бібліотеку для створення таблиці
import pandas as pd

#підключаємо бібліотеку для встановлення пауз під час роботи коду
import time


# КОД ДЛЯ РОБОТИ В СЕРЕДОВИЩІ JUPYTER (треба прибрати решітки, щоб запрацювало):
# import pandas as pd

# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.action_chains import ActionChains

# import time

In [ ]:
# відкриваємо потрібну сторінку
driver.get('http://dfrr.minregion.gov.ua/Projects-list')
driver.page_source

**Пошук кнопки фільтру і її натискання**

In [ ]:
# find_elements_by_id
# find_elements_by_name
# find_elements_by_xpath
# find_element_by_link_text
# find_element_by_partial_link_text
# find_element_by_tag_name
# find_element_by_class_name
# find_element_by_css_selector

In [ ]:
# знаходимо всі фільтри
filter_buttons = driver.find_elements_by_class_name('dropdown-toggle')

# вибираємо першу (нульову) кнопку
regions_button = filter_buttons[0]

# виводимо текст цієї кнопки
print(regions_button.text)

In [ ]:
# створюємо об'єкт, який буде виконувати дії користувача на сторінці
action = ActionChains(driver)

# натискаємо на перший фільтр
regions_button.click()

In [ ]:
# знаходимо всі області і вбираємо Закарпатську (індекс 6)
lis_dropdown = driver.find_elements_by_css_selector('ul.dropdown-menu li')
li_zakarp = lis_dropdown[6]

# вибираємо її у фільтрі
li_zakarp.click()

**Викачка таблиці**

In [ ]:
# назви колонок таблиці
columns = ['Назва проекту', 'Тематичний напрямок', 'Сфера реалізації проекту', 'Заявник', 'Регіон',
          'З ДФРР', 'З місцевих бюджетів', 'Кошти партнерів', 'Загалом', 'Статуси затвердження / реалізації проекту', 
           'Назва генпідрядника', 'Код ЄДРПОУ генпідрядника', 'Рейтингова оцінка', 'Стан реалізації проекту']

# створюємо таблицю з ідентичними колонками до тої, що є на сайті
df = pd.DataFrame(columns = columns)
df

,Назва проекту,Тематичний напрямок,Сфера реалізації проекту,Заявник,Регіон,З ДФРР,З місцевих бюджетів,Кошти партнерів,Загалом,Статуси затвердження / реалізації проекту,Назва генпідрядника,Код ЄДРПОУ генпідрядника,Рейтингова оцінка,Стан реалізації проекту


In [ ]:
# шукаємо таблицю
table = driver.find_element_by_tag_name('table')

# шукаємо рядки таблиці
trs = table.find_elements_by_tag_name('tr')

In [ ]:
# робимо прохід по рядках, починаємо з 2, бо у нульовому і першому рядку немає даних
for i in range(2, len(trs)):
    # створюємо пустий рядок, [len(df)] означає, що ми додаємо в кінець таблиці рядок
    df.loc[len(df)] = None
    
    # шукаємо всі комірки у рядка
    tds = trs[i].find_elements_by_tag_name('td')
    
    # робимо прохід по комірках
    for j in range(len(tds)):
        # у кожну комірку нашої таблиці записуємо відповідку комірку таблиці із сайту,
        # -1 означає, що ми записуємо у останній ряд таблиці, тобто пустий, j - це індекс для проходу по рядках
        df.iloc[-1, j] = tds[j].text
        
df

**Додаємо прохід по сторінках**

In [ ]:
# проходимо від 2 до 5 сторінки включно
for i in range(2, 6):
    # знаходимо посилання на сторінки
    a_pages = driver.find_elements_by_css_selector('div.pager a')
    
    # проходимо по номерах сторінок і шукаємо потрібну наступну
    for a in a_pages:
        if a.text == str(i):
            a.click()
            time.sleep(3)
            
            # ідентичний код до викачки першої сторінки
            table = driver.find_element_by_tag_name('table')
            trs = table.find_elements_by_tag_name('tr')
            
            for i in range(2, len(trs)):
                df.loc[len(df)] = None
                tds = trs[i].find_elements_by_tag_name('td')

                for j in range(len(tds)):
                    df.iloc[-1, j] = tds[j].text
            break
df

**Гортання сторінки**

In [ ]:
time.sleep(3)

driver.execute_script('window.scrollBy(0, 1500)')

**Введення тексту**

In [ ]:
form = driver.find_element_by_tag_name('input')
form.send_keys('школа')
form.submit()

time.sleep(3)
# шукаємо перший результат пошуку
firt_result = driver.find_element_by_css_selector('ul.search_ul li')
print(firt_result.text)

**Керування курсором (у колабі воно не запрацювало для конкретної кнопки, але для інших сайтів/кнопок може запрацювати)**

In [ ]:
action = ActionChains(driver)

button = driver.find_element_by_css_selector('a.btn5')

action.move_to_element(button)
action.perform()